In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import copy
from datetime import datetime, timedelta
# from keras.utils import to_categorical
# import visualkeras
# import tensorflow as tf
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import brier_score_loss
# import optuna
# from optuna.samplers import TPESampler
# import keras
# from keras.callbacks import ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import sys
import os
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import glob 


from sklearn import datasets, ensemble
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.utils import class_weight
import json

import xgboost as xgb
import gc

In [2]:
import os

# Run nvidia-smi to get GPU information
os.system('nvidia-smi')

Mon Dec 23 05:38:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   32C    P0             65W /  500W |       9MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

0

In [3]:
gpu_id = 0

In [4]:
def get_train_val_test_periods(full_df):
    dic_train_val = {}
    dic_test = {}
    
    start_of_test_periods = np.arange(1981,2021,10)
    end_of_test_periods = start_of_test_periods + 9
    
    for iperiod in range(len(start_of_test_periods)):
        df_test_temp = full_df[str(start_of_test_periods[iperiod]):str(end_of_test_periods[iperiod])]
        df_trainval_temp = full_df.drop(df_test_temp.index)
        
        dic_train_val[start_of_test_periods[iperiod]] = df_trainval_temp
        dic_test[start_of_test_periods[iperiod]] = df_test_temp
    return dic_train_val, dic_test

In [5]:
wr_original_series = pd.read_csv('/glade/work/jhayron/Data4Predictability/WR_Series_vSLtest.csv',\
                index_col=0,names=['week0','dist'],skiprows=1,parse_dates=True)
# Rolling window for mode
rolling_mode = (
    wr_original_series.rolling('7d', center=True,min_periods=7)
    .apply(lambda x: x.mode()[0] if not x.mode().empty else float('nan'))
).shift(-3)

# Rolling window for the count of the mode
rolling_mode_count = (
    wr_original_series.rolling('7d', center=True,min_periods=7)
    .apply(lambda x: (x == x.mode()[0]).sum() if not x.mode().empty else 0)
).shift(-3)

# If duration of WR during week was less than 4, assing NO WR class
rolling_mode.loc[rolling_mode_count['week0']<4,'week0'] = 4
wr_series_mode = copy.deepcopy(rolling_mode)
time_index = pd.to_datetime(wr_series_mode.index).dayofweek
wr_series_mode = wr_series_mode.iloc[time_index.isin([0,3])].dropna()
wr_series = copy.deepcopy(wr_series_mode)

for wk in range(2,10):
    series_temp = copy.deepcopy(wr_series["week0"])
    series_temp.index = series_temp.index - timedelta(weeks = wk-1)
    series_temp.name = f'week{wk-1}'
    if wk==2:
        df_shifts = pd.concat([pd.DataFrame(wr_series["week0"]),pd.DataFrame(series_temp)],axis=1)  
    else:
        df_shifts = pd.concat([df_shifts,pd.DataFrame(series_temp)],axis=1)

In [6]:
list_folders = np.sort(glob.glob('/glade/u/home/jhayron/WR_Predictability_v2/1_DataPreparation/PC_Data/PCs/*.csv'))

In [7]:
list_vars = [list_folders[i].split('/')[-1][4:-4] for i in range(len(list_folders))]

In [8]:
for ivar,var in enumerate(list_vars):
    print(ivar,var)

0 IC_SODA
1 IT_SODA
2 MLD_SODA
3 OHC100_SODA
4 OHC200_SODA
5 OHC300_SODA
6 OHC50_SODA
7 OHC700_SODA
8 OLR_ERA5
9 SD_ERA5
10 SSH_SODA
11 SST_OISSTv2
12 SST_SODA
13 STL_1m_ERA5
14 STL_28cm_ERA5
15 STL_7cm_ERA5
16 STL_full_ERA5
17 SWVL_1m_ERA5
18 SWVL_28cm_ERA5
19 SWVL_7cm_ERA5
20 SWVL_full_ERA5
21 U10_ERA5
22 U200_ERA5
23 Z500_ERA5


In [9]:
ivar = 11
list_vars[ivar]

'SST_OISSTv2'

In [10]:
hyperparams = {
    "n_estimators": 100,       # Number of boosting rounds, balances speed and depth
    "max_depth": 6,            # Controls tree depth; deeper trees model complexity better but risk overfitting
    "learning_rate": 0.1,      # Step size shrinkage for reducing overfitting risk
    "subsample": 0.8,          # Fraction of training samples used per tree, helps generalize
    "colsample_bytree": 0.8,   # Fraction of features used per tree, improves robustness
    "gamma": 0,                # Minimum loss reduction to make a split, prevents overfitting
    "min_child_weight": 1,     # Minimum sum of instance weight needed in a leaf, controls complexity
    "reg_alpha": 0,            # L1 regularization term for weights, often fine at 0 unless high sparsity is needed
    "reg_lambda": 1            # L2 regularization term for weights, controls overfitting
}

In [11]:
week_out = 4
print(week_out)
week_out_str = f'week{week_out}'

4


In [12]:
print(list_vars[ivar])

path_weekly_anoms = '/glade/derecho/scratch/jhayron/Data4Predictability/WeeklyAnoms_DetrendedStd_v3_2dg/'
path_nc_anoms = f'{path_weekly_anoms}{list_vars[ivar]}.nc'
anoms = xr.open_dataset(path_nc_anoms)
anoms = anoms.assign_coords(time=pd.DatetimeIndex(anoms.time).normalize())
var_name_nc = list(anoms.data_vars.keys())[0]

# Define a boxcar filter function
def boxcar_filter(data, size):
    kernel = np.ones((size, size)) / (size * size)
    from scipy.signal import convolve2d
    return convolve2d(data, kernel, mode="same", boundary="fill", fillvalue=np.nan)

# Apply boxcar filter
smoothed_anoms = xr.apply_ufunc(
    boxcar_filter,
    anoms,
    kwargs={"size": 3},  # Adjust window size (e.g., 5x5 grid cells)
    input_core_dims=[["lat", "lon"]],
    output_core_dims=[["lat", "lon"]],
    vectorize=True,
)

anoms_flattened = smoothed_anoms[var_name_nc].stack(flat_spatial=('lat', 'lon'))
anoms_flattened_og = copy.deepcopy(anoms_flattened)
# anoms_flattened_og.data[:,anoms_flattened.columns] = anoms_flattened.values WITH THIS LINES I CAN COME BACK
# anoms_flattened_og.unstack('flat_spatial') WITH THIS LINES I CAN COME BACK
anoms_flattened = pd.DataFrame(anoms_flattened,index = anoms_flattened.time)
anoms_flattened = anoms_flattened.dropna(axis=1, how='any')

combined_df = copy.deepcopy(anoms_flattened)
# combined_df['day_sin'] = np.sin(2 * np.pi * combined_df.index.day_of_year / 365)
# combined_df['day_cos'] = np.cos(2 * np.pi * combined_df.index.day_of_year / 365)

SST_OISSTv2


In [13]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()

# Create benchmarks

In [16]:
df_week_0 = df_shifts[['week0']].dropna()

def generate_random_forecast(df_week_0, seed_value=42):
    # Set the random seed for reproducibility
    np.random.seed(seed_value)
    
    # Step 1: Get unique classes and their frequencies
    values = df_week_0['week0'].value_counts()
    
    # Step 2: Calculate the probabilities for each class
    classes = values.index  # Unique classes
    probabilities = values / values.sum()  # Normalize to get probability distribution
    
    # Step 3: Generate a random forecast based on the probabilities
    random_forecast = np.random.choice(classes, size=len(df_week_0), p=probabilities)
    
    # Step 4: Return the random forecast as a DataFrame or Series
    forecast_df = pd.DataFrame(random_forecast, index=df_week_0.index, columns=['y_predicted'])
    
    return forecast_df

def generate_random_forecast_probabilities(df_week_0, seed_value=42):
    # Set the random seed for reproducibility
    np.random.seed(seed_value)
    
    # Step 1: Get unique classes and their frequencies
    values = df_week_0['week0'].value_counts()
    
    # Step 2: Calculate the probabilities for each class
    classes = values.index  # Unique classes
    probabilities = values / values.sum()  # Normalize to get probability distribution
    
    # Step 3: Create a probability forecast for each sample
    # Create a 2D array where each row is the same probability distribution
    prob_matrix = np.tile(probabilities.values, (len(df_week_0), 1))
    
    # Step 4: Return the probability matrix as a DataFrame
    forecast_df = pd.DataFrame(prob_matrix, index=df_week_0.index, columns=classes)[np.arange(5)]
    
    return forecast_df

def generate_random_forecast_with_monthly_probabilities(df_week_0, seed_value=42):
    # Set the random seed for reproducibility
    np.random.seed(seed_value)
    
    # Extract the month from the index (assuming the index is a datetime index)
    df_week_0['month'] = df_week_0.index.month
    
    # Prepare an empty list to store the random forecast
    forecasts = []
    
    # Loop through each month
    for month in range(1, 13):  # Loop through months 1 to 12
        # Filter data for the current month
        month_data = df_week_0[df_week_0['month'] == month]
        
        # Step 1: Get unique classes and their frequencies for the current month
        values = month_data['week0'].value_counts()
        
        # Step 2: Calculate the probabilities for each class in the current month
        classes = values.index  # Unique classes
        probabilities = values / values.sum()  # Normalize to get probability distribution
        
        # Step 3: Generate random forecasts for the current month based on the probabilities
        month_forecast = np.random.choice(classes, size=len(month_data), p=probabilities)
        
        # Store the forecast for the current month
        forecasts.append(pd.Series(month_forecast, index=month_data.index))
    
    # Combine all monthly forecasts into one DataFrame
    forecast_df = pd.concat(forecasts)
    forecast_df = forecast_df.sort_index()  # Sort the index to preserve the original order
    forecast_df = pd.DataFrame(forecast_df,columns=['y_predicted'])
    return forecast_df
    
def generate_probability_forecast_with_monthly_probabilities(df_week_0, seed_value=42):
    # Set the random seed for reproducibility
    np.random.seed(seed_value)
    
    # Extract the month from the index (assuming the index is a datetime index)
    df_week_0['month'] = df_week_0.index.month
    
    # Prepare an empty DataFrame to store the probability forecasts
    all_probabilities = pd.DataFrame(index=df_week_0.index)
    
    # Loop through each month
    for month in range(1, 13):  # Loop through months 1 to 12
        # Filter data for the current month
        month_data = df_week_0[df_week_0['month'] == month]
        
        if month_data.empty:
            continue  # Skip if there's no data for the month
        
        # Step 1: Get unique classes and their frequencies for the current month
        values = month_data['week0'].value_counts()
        
        # Step 2: Calculate the probabilities for each class in the current month
        classes = values.index  # Unique classes
        probabilities = values / values.sum()  # Normalize to get probability distribution
        
        # Step 3: Create a probability matrix for the current month
        prob_matrix = np.tile(probabilities.values, (len(month_data), 1))
        
        # Create a DataFrame for this month's probabilities with appropriate columns
        month_prob_df = pd.DataFrame(prob_matrix, index=month_data.index, columns=classes)
        
        # Append this month's DataFrame to the overall probability DataFrame
        all_probabilities = pd.concat([all_probabilities, month_prob_df])
    
    # Sort the index to match the original order
    all_probabilities = all_probabilities.sort_index()
    
    # Fill missing columns with zeros for months that do not include certain classes
    all_classes = df_week_0['week0'].unique()
    all_probabilities = all_probabilities.reindex(columns=all_classes, fill_value=0).dropna()[np.arange(5)]
    
    return all_probabilities

random_forecast = generate_random_forecast(df_week_0, seed_value=42)
climatology_forecast = generate_random_forecast_with_monthly_probabilities(df_week_0, seed_value=42)
random_forecast_probs = generate_random_forecast_probabilities(df_week_0)
climatology_forecast_probs = generate_probability_forecast_with_monthly_probabilities(df_week_0)

# XGBoost

In [17]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()
dic_trainval, dic_test = get_train_val_test_periods(fully_combined_df)
start_of_test_periods = np.arange(1981,2021,10)

In [18]:
list_results = []
list_results_probs = []

for iperiod in range(len(start_of_test_periods)):
    print(iperiod)
    X_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,-1]

    X_test = dic_test[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_test = dic_test[start_of_test_periods[iperiod]].iloc[:,-1]
    
    cw = class_weight.compute_sample_weight(
        class_weight='balanced',
        y=y_trainval
    )

    model = xgb.XGBClassifier(n_estimators=100,
                            max_depth=hyperparams['max_depth'],
                            learning_rate=hyperparams['learning_rate'],
                            subsample=hyperparams['subsample'],
                            colsample_bytree=hyperparams['colsample_bytree'],
                            # colsample_bylevel=best_params['colsample_bylevel'],
                            gamma=hyperparams['gamma'],
                            reg_alpha=hyperparams['reg_alpha'],
                            reg_lambda=hyperparams['reg_lambda'],
                            num_class=5,
                            objective = "multi:softprob",
                            tree_method='hist',
                            device = f'cuda:{gpu_id}')
    model.fit(X_trainval, y_trainval, sample_weight=cw)
    y_predicted = model.predict(X_test)
    y_predicted_probs = model.predict_proba(X_test)
    y_predicted_probs = pd.DataFrame(y_predicted_probs,index=y_test.index)
    df_results_temp = pd.DataFrame(np.array([y_test.values,y_predicted]).T,
                                   index=y_test.index,
                                   columns=['y_true','y_predicted'])
    list_results.append(df_results_temp)
    list_results_probs.append(y_predicted_probs)
    
df_results_full = pd.concat(list_results,axis=0)
df_results_full_probs = pd.concat(list_results_probs,axis=0)

0


/glade/work/jhayron/conda-envs/pytorch_wr/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [05:41:31] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1730232887822/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


1
2
3


In [19]:
f1_results = f1_score(df_results_full['y_true'],df_results_full['y_predicted'],average='micro')

f1_random = f1_score(df_results_full['y_true'],
         random_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')

persistence_forecast = df_week_0.loc[df_results_full['y_true'].index]
f1_persistence = f1_score(df_results_full['y_true'],
         persistence_forecast['week0'].loc[df_results_full['y_true'].index],average='micro')

f1_climatology = f1_score(df_results_full['y_true'],
         climatology_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')
print(f'F1 XGBoost:', f1_results)
print('F1 Random:',f1_random)
print('F1 Persistence:',f1_persistence)
print('F1 Climatology:',f1_climatology)

F1 XGBoost: 0.2721055202735711
F1 Random: 0.21812408402540304
F1 Persistence: 0.23326819736199317
F1 Climatology: 0.23277967757694187


# ANN

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split


In [21]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()
dic_trainval, dic_test = get_train_val_test_periods(fully_combined_df)
start_of_test_periods = np.arange(1981,2021,10)

In [22]:
# Define general-purpose ANN model
class GeneralANN(nn.Module):
    def __init__(self, input_size, num_classes):
        self.do = 0.3
        super(GeneralANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),      # First hidden layer with fewer neurons
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(self.do),                  # Dropout for regularization
            
            nn.Linear(1024, 512),             # Second hidden layer
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(self.do),
            
            nn.Linear(512, 256),              # Third hidden layer (optional, depending on needs)
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(self.do),

            nn.Linear(256, 64),              # Fourth hidden layer (optional, depending on needs)
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(self.do),
            
            nn.Linear(64, num_classes)       # Output layer for classification
        )

    def forward(self, x):
        return self.model(x)
        
def build_train_evaluate_ann(X_trainval,y_trainval,X_test,y_test):
    # Split the dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Convert split data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)
    
    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    batch_size = 128
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, loss function, and optimizer
    input_size = X_trainval.shape[1]      # Number of features
    num_classes = 5         # Adjust based on your classification task
    model = GeneralANN(input_size, num_classes)
    
    # Move model to GPU if available
    model = model.to(device)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss() # Appropriate for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=0.00001)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
        
    # Early stopping parameters
    epochs = 50
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop with early stopping
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        # Average losses
        train_loss = running_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
        # print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        
        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'ann_best_model.pth')  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print("Early stopping triggered")
                break
    
    # Load the best model after training
    model.load_state_dict(torch.load('ann_best_model.pth'))
    os.remove('ann_best_model.pth')
    y_predicted = model(X_test_tensor)
    probabilities = torch.softmax(y_predicted, dim=1)
    y_predicted = torch.argmax(y_predicted, dim=1)
    
    results_temp = pd.DataFrame(y_predicted.cpu().detach().numpy(), index=y_test.index, columns=['y_predicted'])
    results_temp_probabilities = pd.DataFrame(probabilities.cpu().detach().numpy(), index=y_test.index)
    return results_temp, results_temp_probabilities

In [23]:
list_results = []
list_results_probs = []

for iperiod in range(len(start_of_test_periods)):
    print(iperiod)
    X_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,-1]

    X_test = dic_test[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_test = dic_test[start_of_test_periods[iperiod]].iloc[:,-1]

    
    y_predicted, y_predicted_probs = build_train_evaluate_ann(X_trainval,y_trainval,X_test,y_test)
    df_results_temp = pd.DataFrame(np.array([y_test.values,y_predicted['y_predicted'].values]).T,
                                   index=y_test.index,
                                   columns=['y_true','y_predicted'])
    list_results.append(df_results_temp)
    list_results_probs.append(y_predicted_probs)
    
df_results_full = pd.concat(list_results,axis=0)
df_results_full_probs = pd.concat(list_results_probs,axis=0)

0


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3264798996.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ann_best_model.pth'))


1
2
3


In [24]:
f1_results = f1_score(df_results_full['y_true'],df_results_full['y_predicted'],average='micro')

f1_random = f1_score(df_results_full['y_true'],
         random_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')

persistence_forecast = df_week_0.loc[df_results_full['y_true'].index]
f1_persistence = f1_score(df_results_full['y_true'],
         persistence_forecast['week0'].loc[df_results_full['y_true'].index],average='micro')

f1_climatology = f1_score(df_results_full['y_true'],
         climatology_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')
print(f'F1 ANN:', f1_results)
print('F1 Random:',f1_random)
print('F1 Persistence:',f1_persistence)
print('F1 Climatology:',f1_climatology)

F1 ANN: 0.2122618466047875
F1 Random: 0.21812408402540304
F1 Persistence: 0.23326819736199317
F1 Climatology: 0.23277967757694187


# ANN Weights

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split


In [26]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()
dic_trainval, dic_test = get_train_val_test_periods(fully_combined_df)
start_of_test_periods = np.arange(1981,2021,10)

In [27]:
# Define general-purpose ANN model
class GeneralANN(nn.Module):
    def __init__(self, input_size, num_classes):
        self.do = 0.3
        super(GeneralANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),      # First hidden layer with fewer neurons
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(self.do),                  # Dropout for regularization
            
            nn.Linear(1024, 512),             # Second hidden layer
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(self.do),
            
            nn.Linear(512, 256),              # Third hidden layer (optional, depending on needs)
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(self.do),

            nn.Linear(256, 64),              # Fourth hidden layer (optional, depending on needs)
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(self.do),
            
            nn.Linear(64, num_classes)       # Output layer for classification
        )

    def forward(self, x):
        return self.model(x)
        
def build_train_evaluate_ann(X_trainval,y_trainval,X_test,y_test):
    # Split the dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=False)
    cw_classes = compute_class_weight(class_weight='balanced', classes=np.arange(5), y=y_trainval)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    cw_classes_tensor = torch.tensor(cw_classes, dtype=torch.float32, device=device)
    
    # Convert split data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)
    
    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    batch_size = 128
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model, loss function, and optimizer
    input_size = X_trainval.shape[1]      # Number of features
    num_classes = 5         # Adjust based on your classification task
    model = GeneralANN(input_size, num_classes)
    
    # Move model to GPU if available
    model = model.to(device)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss(weight = cw_classes_tensor) # Appropriate for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=0.00001)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
        
    # Early stopping parameters
    epochs = 50
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop with early stopping
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        # Average losses
        train_loss = running_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
        # print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        
        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'ann_best_model.pth')  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print("Early stopping triggered")
                break
    
    # Load the best model after training
    model.load_state_dict(torch.load('ann_best_model.pth'))
    os.remove('ann_best_model.pth')
    y_predicted = model(X_test_tensor)
    probabilities = torch.softmax(y_predicted, dim=1)
    y_predicted = torch.argmax(y_predicted, dim=1)
    
    results_temp = pd.DataFrame(y_predicted.cpu().detach().numpy(), index=y_test.index, columns=['y_predicted'])
    results_temp_probabilities = pd.DataFrame(probabilities.cpu().detach().numpy(), index=y_test.index)
    return results_temp, results_temp_probabilities

In [28]:
list_results = []
list_results_probs = []

for iperiod in range(len(start_of_test_periods)):
    print(iperiod)
    X_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,-1]

    X_test = dic_test[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_test = dic_test[start_of_test_periods[iperiod]].iloc[:,-1]

    
    y_predicted, y_predicted_probs = build_train_evaluate_ann(X_trainval,y_trainval,X_test,y_test)
    df_results_temp = pd.DataFrame(np.array([y_test.values,y_predicted['y_predicted'].values]).T,
                                   index=y_test.index,
                                   columns=['y_true','y_predicted'])
    list_results.append(df_results_temp)
    list_results_probs.append(y_predicted_probs)
    
df_results_full = pd.concat(list_results,axis=0)
df_results_full_probs = pd.concat(list_results_probs,axis=0)

0


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/416893788.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ann_best_model.pth'))


1


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/416893788.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ann_best_model.pth'))


2


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/416893788.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ann_best_model.pth'))


3


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/416893788.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ann_best_model.pth'))


In [29]:
f1_results = f1_score(df_results_full['y_true'],df_results_full['y_predicted'],average='micro')

f1_random = f1_score(df_results_full['y_true'],
         random_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')

persistence_forecast = df_week_0.loc[df_results_full['y_true'].index]
f1_persistence = f1_score(df_results_full['y_true'],
         persistence_forecast['week0'].loc[df_results_full['y_true'].index],average='micro')

f1_climatology = f1_score(df_results_full['y_true'],
         climatology_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')
print(f'F1 ANN:', f1_results)
print('F1 Random:',f1_random)
print('F1 Persistence:',f1_persistence)
print('F1 Climatology:',f1_climatology)

F1 ANN: 0.22716170004885197
F1 Random: 0.21812408402540304
F1 Persistence: 0.23326819736199317
F1 Climatology: 0.23277967757694187


# LSTM

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn


In [31]:
def create_lagged_dataset(X, y, num_lags=4, skip=1):
    """
    Creates a dataset with lagged data for LSTM training.
    X: Original feature array (samples, features).
    y: Original target array (samples,).
    num_lags: Number of lags to include (how many previous time steps).
    skip: The step size to skip between lags (e.g., skip=2 means t-8, t-6, t-4, t-2, t=0).
    Returns: X_lagged (samples, timesteps, features), y_lagged (samples,).
    """
    samples, features = X.shape
    X_lagged = []
    y_lagged = []
    
    # Create sequences with lagged data considering skip
    for i in range(num_lags * skip, samples):
        # Select data points based on the skip factor
        lagged_sequence = X[i - num_lags * skip:i:skip]  # (num_lags, features)
        lagged_sequence = np.vstack([lagged_sequence, X[i]])  # Add t=0 at the end
        X_lagged.append(lagged_sequence)
        y_lagged.append(y[i])  # Target corresponds to t=0
    
    # Convert to arrays with the shape (samples, timesteps, features)
    X_lagged = np.array(X_lagged)
    y_lagged = np.array(y_lagged)
    
    return X_lagged, y_lagged

In [32]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_lstm_layers=2, dropout=0.3):
        super(LSTMModel, self).__init__()
        
        # Define the LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_lstm_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to map LSTM outputs to class scores
        self.fc = nn.Linear(hidden_size, num_classes)
        
        # Dropout layer for regularization
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Forward pass through LSTM
        lstm_out, (hn, cn) = self.lstm(x)
        
        # We use the output of the last time step (t=0 in this case)
        last_time_step_output = lstm_out[:, -1, :]
        
        # Pass the output through the fully connected layer
        out = self.fc(last_time_step_output)
        
        # Apply dropout regularization
        out = self.dropout(out)
        
        return out

In [33]:
def build_train_evaluate_lstm(X_trainval,y_trainval,X_test,y_test):
    # Create the lagged dataset
    num_lags = 3
    skip = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    X_trainval_lagged, y_trainval_lagged = create_lagged_dataset(X_trainval, y_trainval.values, num_lags=num_lags, skip=skip)
    X_test_lagged, y_test_lagged = create_lagged_dataset(X_test, y_test.values, num_lags=num_lags, skip=skip)

    # Convert to PyTorch tensors
    X_test_tensor = torch.tensor(X_test_lagged, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test_lagged, dtype=torch.long).to(device)
    
    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_trainval_lagged, y_trainval_lagged, test_size=0.2, shuffle=False)
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
    
    # Create datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
        
    # Create data loaders
    batch_size = 128
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Example initialization and usage:
    input_size = X_train_tensor.shape[-1]  # Number of features (size of each input vector)
    hidden_size = 512  # Hidden state size of LSTM
    num_classes = 5  # Number of classes for classification
    num_lstm_layers = 2  # Number of LSTM layers
    dropout = 0.2  # Dropout rate
    
    # Instantiate the model
    model = LSTMModel(input_size, hidden_size, num_classes, num_lstm_layers, dropout)
    
    # Print the model architecture
    model = model.to(device)
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=0.00001)

    # Early stopping parameters
    epochs = 50
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop with early stopping
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device (GPU/CPU)
            
            optimizer.zero_grad()  # Zero out the gradients
            outputs = model(inputs)  # Get predictions from the model
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model weights
            
            running_loss += loss.item()  # Track loss for this batch
    
        # Validation phase (no gradients needed)
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        y_val_pred = []
        y_val_true = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move to device
                outputs = model(inputs)  # Get predictions
                loss = criterion(outputs, labels)  # Compute loss
                val_loss += loss.item()  # Track validation loss
                y_val_pred.append(outputs.argmax(dim=1))  # Get predicted class labels
                y_val_true.append(labels)  # True labels
    
        # Average losses
        train_loss = running_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
    
        # Convert validation predictions to a single tensor
        y_val_pred = torch.cat(y_val_pred).cpu().numpy()
        y_val_true = torch.cat(y_val_true).cpu().numpy()
    
        # Evaluate F1 score (or any other metric you prefer)
        f1 = f1_score(y_val_true, y_val_pred, average='micro')
        # print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, F1 Score: {f1:.4f}")
    
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_lstm_model.pth')  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print("Early stopping triggered")
                break
    
    # Load the best model after training
    model.load_state_dict(torch.load('best_lstm_model.pth'))
    os.remove('best_lstm_model.pth')
    
    y_predicted = model(X_test_tensor)
    probabilities = torch.softmax(y_predicted, dim=1)
    y_predicted = torch.argmax(y_predicted, dim=1)
    
    results_temp = pd.DataFrame(y_predicted.cpu().detach().numpy(), 
                                index=y_test[num_lags*skip:].index, columns=['y_predicted'])
    results_temp_probabilities = pd.DataFrame(probabilities.cpu().detach().numpy(),
                                              index=y_test[num_lags*skip:].index)
    return y_test[num_lags*skip:], results_temp, results_temp_probabilities

In [34]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()
dic_trainval, dic_test = get_train_val_test_periods(fully_combined_df)
start_of_test_periods = np.arange(1981,2021,10)

In [35]:
list_results = []
list_ytests_modified = []
list_results_probs = []

for iperiod in range(len(start_of_test_periods)):
    print(iperiod)
    X_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,-1]

    X_test = dic_test[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_test = dic_test[start_of_test_periods[iperiod]].iloc[:,-1]
    
    y_test_new, y_predicted, y_predicted_probs  = build_train_evaluate_lstm(X_trainval,y_trainval,
                                                        X_test,y_test)
    
    df_results_temp = pd.DataFrame(np.array([y_test_new.values,y_predicted['y_predicted'].values]).T,
                                   index=y_test_new.index,
                                   columns=['y_true','y_predicted'])
    list_results.append(df_results_temp)
    list_results_probs.append(y_predicted_probs)
    
df_results_full = pd.concat(list_results,axis=0)
df_results_full_probs = pd.concat(list_results_probs,axis=0)

0


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3909201925.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

1


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3909201925.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

2


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3909201925.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

3


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3909201925.py:112: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

In [36]:
f1_results = f1_score(df_results_full['y_true'],df_results_full['y_predicted'],average='micro')

f1_random = f1_score(df_results_full['y_true'],
         random_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')

persistence_forecast = df_week_0.loc[df_results_full['y_true'].index]
f1_persistence = f1_score(df_results_full['y_true'],
         persistence_forecast['week0'].loc[df_results_full['y_true'].index],average='micro')

f1_climatology = f1_score(df_results_full['y_true'],
         climatology_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')
print(f'F1 LSTM:', f1_results)
print('F1 Random:',f1_random)
print('F1 Persistence:',f1_persistence)
print('F1 Climatology:',f1_climatology)

F1 LSTM: 0.25896805896805897
F1 Random: 0.2176904176904177
F1 Persistence: 0.2339066339066339
F1 Climatology: 0.23267813267813267


# LSTM Weights

In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn


In [38]:
def create_lagged_dataset(X, y, num_lags=4, skip=1):
    """
    Creates a dataset with lagged data for LSTM training.
    X: Original feature array (samples, features).
    y: Original target array (samples,).
    num_lags: Number of lags to include (how many previous time steps).
    skip: The step size to skip between lags (e.g., skip=2 means t-8, t-6, t-4, t-2, t=0).
    Returns: X_lagged (samples, timesteps, features), y_lagged (samples,).
    """
    samples, features = X.shape
    X_lagged = []
    y_lagged = []
    
    # Create sequences with lagged data considering skip
    for i in range(num_lags * skip, samples):
        # Select data points based on the skip factor
        lagged_sequence = X[i - num_lags * skip:i:skip]  # (num_lags, features)
        lagged_sequence = np.vstack([lagged_sequence, X[i]])  # Add t=0 at the end
        X_lagged.append(lagged_sequence)
        y_lagged.append(y[i])  # Target corresponds to t=0
    
    # Convert to arrays with the shape (samples, timesteps, features)
    X_lagged = np.array(X_lagged)
    y_lagged = np.array(y_lagged)
    
    return X_lagged, y_lagged

In [39]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_lstm_layers=2, dropout=0.3):
        super(LSTMModel, self).__init__()
        
        # Define the LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_lstm_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer to map LSTM outputs to class scores
        self.fc = nn.Linear(hidden_size, num_classes)
        
        # Dropout layer for regularization
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Forward pass through LSTM
        lstm_out, (hn, cn) = self.lstm(x)
        
        # We use the output of the last time step (t=0 in this case)
        last_time_step_output = lstm_out[:, -1, :]
        
        # Pass the output through the fully connected layer
        out = self.fc(last_time_step_output)
        
        # Apply dropout regularization
        out = self.dropout(out)
        
        return out

In [40]:
def build_train_evaluate_lstm(X_trainval,y_trainval,X_test,y_test):
    # Create the lagged dataset
    num_lags = 3
    skip = 2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    cw_classes = compute_class_weight(class_weight='balanced', classes=np.arange(5), y=y_trainval)
    cw_classes_tensor = torch.tensor(cw_classes, dtype=torch.float32, device=device)
    
    X_trainval_lagged, y_trainval_lagged = create_lagged_dataset(X_trainval, y_trainval.values, num_lags=num_lags, skip=skip)
    X_test_lagged, y_test_lagged = create_lagged_dataset(X_test, y_test.values, num_lags=num_lags, skip=skip)

    # Convert to PyTorch tensors
    X_test_tensor = torch.tensor(X_test_lagged, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test_lagged, dtype=torch.long).to(device)
    
    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_trainval_lagged, y_trainval_lagged, test_size=0.2, shuffle=False)
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
    
    # Create datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
        
    # Create data loaders
    batch_size = 128
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Example initialization and usage:
    input_size = X_train_tensor.shape[-1]  # Number of features (size of each input vector)
    hidden_size = 512  # Hidden state size of LSTM
    num_classes = 5  # Number of classes for classification
    num_lstm_layers = 2  # Number of LSTM layers
    dropout = 0.2  # Dropout rate
    
    # Instantiate the model
    model = LSTMModel(input_size, hidden_size, num_classes, num_lstm_layers, dropout)
    
    # Print the model architecture
    model = model.to(device)
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss(weight = cw_classes_tensor)  # Suitable for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=0.00001)

    # Early stopping parameters
    epochs = 50
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop with early stopping
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device (GPU/CPU)
            
            optimizer.zero_grad()  # Zero out the gradients
            outputs = model(inputs)  # Get predictions from the model
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model weights
            
            running_loss += loss.item()  # Track loss for this batch
    
        # Validation phase (no gradients needed)
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        y_val_pred = []
        y_val_true = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move to device
                outputs = model(inputs)  # Get predictions
                loss = criterion(outputs, labels)  # Compute loss
                val_loss += loss.item()  # Track validation loss
                y_val_pred.append(outputs.argmax(dim=1))  # Get predicted class labels
                y_val_true.append(labels)  # True labels
    
        # Average losses
        train_loss = running_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
    
        # Convert validation predictions to a single tensor
        y_val_pred = torch.cat(y_val_pred).cpu().numpy()
        y_val_true = torch.cat(y_val_true).cpu().numpy()
    
        # Evaluate F1 score (or any other metric you prefer)
        f1 = f1_score(y_val_true, y_val_pred, average='micro')
        # print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, F1 Score: {f1:.4f}")
    
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_lstm_model.pth')  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print("Early stopping triggered")
                break
    
    # Load the best model after training
    model.load_state_dict(torch.load('best_lstm_model.pth'))
    os.remove('best_lstm_model.pth')
    
    y_predicted = model(X_test_tensor)
    probabilities = torch.softmax(y_predicted, dim=1)
    y_predicted = torch.argmax(y_predicted, dim=1)
    
    results_temp = pd.DataFrame(y_predicted.cpu().detach().numpy(), 
                                index=y_test[num_lags*skip:].index, columns=['y_predicted'])
    results_temp_probabilities = pd.DataFrame(probabilities.cpu().detach().numpy(),
                                              index=y_test[num_lags*skip:].index)
    return y_test[num_lags*skip:], results_temp, results_temp_probabilities

In [41]:
fully_combined_df = pd.concat([combined_df,df_shifts[week_out_str]],axis=1)
fully_combined_df = fully_combined_df.dropna()
dic_trainval, dic_test = get_train_val_test_periods(fully_combined_df)
start_of_test_periods = np.arange(1981,2021,10)

In [42]:
list_results = []
list_ytests_modified = []
list_results_probs = []

for iperiod in range(len(start_of_test_periods)):
    print(iperiod)
    X_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_trainval = dic_trainval[start_of_test_periods[iperiod]].iloc[:,-1]

    X_test = dic_test[start_of_test_periods[iperiod]].iloc[:,:-1].values
    y_test = dic_test[start_of_test_periods[iperiod]].iloc[:,-1]
    
    y_test_new, y_predicted, y_predicted_probs  = build_train_evaluate_lstm(X_trainval,y_trainval,
                                                        X_test,y_test)
    
    df_results_temp = pd.DataFrame(np.array([y_test_new.values,y_predicted['y_predicted'].values]).T,
                                   index=y_test_new.index,
                                   columns=['y_true','y_predicted'])
    list_results.append(df_results_temp)
    list_results_probs.append(y_predicted_probs)
    
df_results_full = pd.concat(list_results,axis=0)
df_results_full_probs = pd.concat(list_results_probs,axis=0)

0


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3939569735.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

1


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3939569735.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

2


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3939569735.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

3


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3939569735.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_lstm_model.pth'))

In [43]:
f1_results = f1_score(df_results_full['y_true'],df_results_full['y_predicted'],average='micro')

f1_random = f1_score(df_results_full['y_true'],
         random_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')

persistence_forecast = df_week_0.loc[df_results_full['y_true'].index]
f1_persistence = f1_score(df_results_full['y_true'],
         persistence_forecast['week0'].loc[df_results_full['y_true'].index],average='micro')

f1_climatology = f1_score(df_results_full['y_true'],
         climatology_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')
print(f'F1 LSTM:', f1_results)
print('F1 Random:',f1_random)
print('F1 Persistence:',f1_persistence)
print('F1 Climatology:',f1_climatology)

F1 LSTM: 0.22997542997542997
F1 Random: 0.2176904176904177
F1 Persistence: 0.2339066339066339
F1 Climatology: 0.23267813267813267


# CNN

In [44]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F

In [45]:
def get_train_val_test_periods_cnn(X,y):
    dic_train_val = {}
    dic_test = {}
    
    start_of_test_periods = np.arange(1981,2021,10)
    end_of_test_periods = start_of_test_periods + 9
    
    for iperiod in range(len(start_of_test_periods)):
        X_test_temp = X.sel(time=slice(str(start_of_test_periods[iperiod]),str(end_of_test_periods[iperiod])))
        X_trainval_temp = X.sel(time = ~X['time'].isin(X_test_temp.time))
    
        y_test_temp = y[str(start_of_test_periods[iperiod]):str(end_of_test_periods[iperiod])]
        y_trainval_temp = y.drop(y_test_temp.index)
        
        dic_train_val[start_of_test_periods[iperiod]] = (X_trainval_temp,y_trainval_temp)
        dic_test[start_of_test_periods[iperiod]] = (X_test_temp,y_test_temp)
    return dic_train_val, dic_test

In [46]:

class CustomCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(CustomCNN, self).__init__()
        self.out_first_channel = 128
        self.kernel_size_first_channel = 3
        self.kernel_size_second_channel = 3
        self.kernel_size_third_channel = 3
        self.stride = 1
        self.padding = "valid"
        self.ks_maxpool = 4
        self.stride_maxpool = 2
        self.dropout_rate = 0.1
        
        # First convolutional block
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=self.out_first_channel, 
                               kernel_size=self.kernel_size_first_channel, stride=self.stride, padding=self.padding)
        self.bn1 = nn.BatchNorm2d(self.out_first_channel)
        self.pool1 = nn.MaxPool2d(kernel_size=self.ks_maxpool, stride=self.stride_maxpool)
        self.dropout1 = nn.Dropout(self.dropout_rate)
        
        # Second convolutional block
        self.conv2 = nn.Conv2d(in_channels=self.out_first_channel, out_channels=self.out_first_channel*2, 
                               kernel_size=self.kernel_size_second_channel, stride=self.stride, padding=self.padding)
        self.bn2 = nn.BatchNorm2d(self.out_first_channel*2)
        self.pool2 = nn.MaxPool2d(kernel_size=self.ks_maxpool, stride=self.stride_maxpool)
        self.dropout2 = nn.Dropout(self.dropout_rate)
        
        # Third convolutional block
        self.conv3 = nn.Conv2d(in_channels=self.out_first_channel*2, out_channels=self.out_first_channel*4, 
                               kernel_size=self.kernel_size_third_channel, stride=self.stride, padding=self.padding)
        self.bn3 = nn.BatchNorm2d(self.out_first_channel*4)
        self.pool3 = nn.MaxPool2d(kernel_size=self.ks_maxpool, stride=self.stride_maxpool)
        self.dropout3 = nn.Dropout(self.dropout_rate)
        
        # Global average pooling layer
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layer
        self.fc1 = nn.Linear(self.out_first_channel*4, self.out_first_channel*2)
        self.fc2 = nn.Linear(self.out_first_channel*2, self.out_first_channel)
        self.fc3 = nn.Linear(self.out_first_channel, num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        # First block: Conv -> BatchNorm -> ReLU -> MaxPool
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        # Second block: Conv -> BatchNorm -> ReLU -> MaxPool
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # Third block: Conv -> BatchNorm -> ReLU -> MaxPool
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = self.dropout3(x)
        
        # Global average pooling
        x = self.global_avg_pool(x)
        
        # Flatten for the fully connected layers
        x = torch.flatten(x, 1)
        
        # Fully connected layers
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [47]:
def build_train_evaluate_cnn(X_trainval,y_trainval,X_test,y_test):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    X_trainval = X_trainval.expand_dims("channel", axis=1)
    X_test = X_test.expand_dims("channel", axis=1)
    
    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, shuffle=False)
    
    # Convert to PyTorch tensors
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.long).to(device)
    
    # Create datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
    
    # Create data loaders
    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Instantiate the model and move to the device
    model = CustomCNN(num_classes=5).to(device)
    
    criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Early stopping parameters
    epochs = 50
    patience = 5
    best_val_loss = float('inf')
    patience_counter = 0
    
    # Training loop with early stopping
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device (GPU/CPU)
            
            optimizer.zero_grad()  # Zero out the gradients
            outputs = model(inputs)  # Get predictions from the model
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update model weights
            
            running_loss += loss.item()  # Track loss for this batch
    
        # Validation phase (no gradients needed)
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        y_val_pred = []
        y_val_true = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move to device
                outputs = model(inputs)  # Get predictions
                loss = criterion(outputs, labels)  # Compute loss
                val_loss += loss.item()  # Track validation loss
                y_val_pred.append(outputs.argmax(dim=1))  # Get predicted class labels
                y_val_true.append(labels)  # True labels
    
        # Average losses
        train_loss = running_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
    
        # Convert validation predictions to a single tensor
        y_val_pred = torch.cat(y_val_pred).cpu().numpy()
        y_val_true = torch.cat(y_val_true).cpu().numpy()
    
        # Evaluate F1 score (or any other metric you prefer)
        f1 = f1_score(y_val_true, y_val_pred, average='micro')
        # print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, F1 Score: {f1:.4f}")
        torch.save(model.state_dict(), 'best_cnn_model.pth')
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_cnn_model.pth')  # Save the best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print("Early stopping triggered")
                break
    
    # Load the best model after training
    model.load_state_dict(torch.load('best_cnn_model.pth'))
    os.remove('best_cnn_model.pth')

    y_predicted = model(X_test_tensor)
    probabilities = torch.softmax(y_predicted, dim=1)
    y_predicted = torch.argmax(y_predicted, dim=1)
    
    results_temp = pd.DataFrame(y_predicted.cpu().detach().numpy(), index=y_test.index, columns=['y_predicted'])
    results_temp_probabilities = pd.DataFrame(probabilities.cpu().detach().numpy(),
                                              index=y_test.index)
    return results_temp, results_temp_probabilities

In [48]:
timeanoms = anoms.time.values
timewrs = df_shifts[week_out_str].dropna().index.values
time_intersection = np.intersect1d(timeanoms, timewrs)

X = anoms[var_name_nc].sel(time=time_intersection)
y = df_shifts[week_out_str].loc[time_intersection]

In [49]:
dic_trainval, dic_test = get_train_val_test_periods_cnn(X,y)
start_of_test_periods = np.arange(1981,2021,10)

In [50]:
list_results = []
list_results_probs = []

for iperiod in range(len(start_of_test_periods)):
    print(iperiod)
    X_trainval = dic_trainval[start_of_test_periods[iperiod]][0]
    y_trainval = dic_trainval[start_of_test_periods[iperiod]][1]

    X_test = dic_test[start_of_test_periods[iperiod]][0]
    y_test = dic_test[start_of_test_periods[iperiod]][1]

    y_predicted, y_predicted_probs = build_train_evaluate_cnn(X_trainval,y_trainval,X_test,y_test)
    
    df_results_temp = pd.DataFrame(np.array([y_test.values,y_predicted['y_predicted'].values]).T,
                                   index=y_test.index,
                                   columns=['y_true','y_predicted'])
    
    list_results.append(df_results_temp)
    list_results_probs.append(y_predicted_probs)
    
    # Collect garbage to release memory
    gc.collect()
    
    # Clear PyTorch cache
    torch.cuda.empty_cache()
    
df_results_full = pd.concat(list_results,axis=0)
df_results_full_probs = pd.concat(list_results_probs,axis=0)

0


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3032166677.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_model.pth'))


1


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3032166677.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_model.pth'))


2


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3032166677.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_model.pth'))


3


/glade/derecho/scratch/jhayron/tmp/ipykernel_174869/3032166677.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_model.pth'))


In [51]:
f1_results = f1_score(df_results_full['y_true'],df_results_full['y_predicted'],average='micro')

f1_random = f1_score(df_results_full['y_true'],
         random_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')

persistence_forecast = df_week_0.loc[df_results_full['y_true'].index]
f1_persistence = f1_score(df_results_full['y_true'],
         persistence_forecast['week0'].loc[df_results_full['y_true'].index],average='micro')

f1_climatology = f1_score(df_results_full['y_true'],
         climatology_forecast['y_predicted'].loc[df_results_full['y_true'].index],average='micro')
print(f'F1 CNN:', f1_results)
print('F1 Random:',f1_random)
print('F1 Persistence:',f1_persistence)
print('F1 Climatology:',f1_climatology)

F1 CNN: 0.2340009770395701
F1 Random: 0.21812408402540304
F1 Persistence: 0.23326819736199317
F1 Climatology: 0.23277967757694187
